<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/Claude/Claude102_SQLQueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install anthropic -q
!pip install supabase -q
!pip install pyodbc -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.7/334.7 kB 5.4 MB/s eta 0:00:00


In [6]:
import anthropic

from google.colab import userdata
claude_api_key=userdata.get('Claude_api_key')

client = anthropic.Anthropic(api_key=claude_api_key)

In [7]:
#
# Retrieve a file from Github Repository

def get_ChinookDB_ER_from_github(file_url):

  # Send a GET request to the file URL
  response = requests.get(file_url)

  # Check if the request was successful
  if response.status_code == 200:
      # Get the content of the file
      file_content = response.text
      return file_content
  else:
      return(f'Failed to retrieve the file: {response.status_code}')

In [11]:
from supabase import create_client, Client

In [94]:
from supabase import create_client, Client
from sqlalchemy.sql import text

from sqlalchemy import create_engine
# Test connection to Supabase using Supabase client

# Make sure the connection string is a proper string within quotes
supabase_conn_str = userdata.get('supabase_conn_str')
#supabase_conn_str = "postgresql+psycopg2://[USERID:PASSWORD]@[HOSTNAME:PORT_NUMBER]/[DATABASE_NAME]"


# Use the connection string with SQLAlchemy's create_engine function
db_engine = create_engine(supabase_conn_str)
sql_conn = db_engine.connect()

# test the connection
result = sql_conn.execute(text('select * from "Customer";'))
df = pd.DataFrame(result)
df.head(3)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


In [62]:
import requests
chinookDB_ER_from_github = get_ChinookDB_ER_from_github("https://raw.githubusercontent.com/cooolbabu/GoogleGemini101/main/OpenAI/ChinookDB_ER.txt")
# Markdown(chinookDB_ER_from_github) ## uncoment to view file from Github

## Build the prompt
dba_instructions="""
1. You are a Data Architect for ChinookDB.
2. An entiry-relationship diagram for ChinookDB, in the form of a Mermaid.js script is provide to you
3. You will use this ER diagram to deliver SQL queries.
4. Database dialect is PostgreSQL
5. Use quotes for database object names in SQL statements
6. Give your response in JSON format having two attributes. SQL and Explanation
7. SQL attribute will have the value of the generated SQL code
8. Explanation attribute will have the value of the explanation
9. Remove all introductory messages.

"""

In [63]:
Prompt = dba_instructions + chinookDB_ER_from_github + "Get total sales for each genre by quarter for each year "
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": Prompt}
    ]
)


In [ ]:
pd.set_option('display.max_rows', None)
df.sort_values(by=['Year', 'Quarter', 'Genre'])

In [ ]:
Prompt = "Write a Python code for typical data engineering workflow."


completion = client.messages.stream(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": Prompt,
        }
    ],
    model="claude-3-opus-20240229",
)

with completion as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Here's a Python code that demonstrates a typical data engineering workflow:

```python
import pandas as pd
from sqlalchemy import create_engine

# Extract data from a source (e.g., CSV file)
def extract_data(file_path):
    data = pd.read_csv(file_path)
    return data

# Transform the data (e.g., cleaning, filtering, aggregating)
def transform_data(data):
    # Remove duplicates
    data.drop_duplicates(inplace=True)
    
    # Filter data based on a condition
    filtered_data = data[data['column_name'] > 0]
    
    # Aggregate data
    aggregated_data = filtered_data.groupby('column_name').sum()
    
    return aggregated_data

# Load the transformed data into a target system (e.g., database)
def load_data(data, db_url, table_name):
    engine = create_engine(db_url)
    data.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Data loaded into {table_name} table.")

# Main function to orchestrate the workflow
def main():
    # Extract
    file_path = 'path/to/y

In [ ]:
import asyncio
from anthropic import AsyncAnthropic

async_client = AsyncAnthropic(api_key=claude_api_key)

Prompt = "Could you explain what MLOps is and suggest some of the best tools to use for it?"

async def main() -> None:

    message = await async_client.messages.create(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": Prompt,
            }
        ],
        model="claude-3-opus-20240229",
    )
    display(Markdown(message.content[0].text))


In [ ]:
# asyncio.run(main())
await main()  # Juypter notebook

Certainly! MLOps, short for Machine Learning Operations, is a set of practices, processes, and tools that aim to streamline the development, deployment, and maintenance of machine learning models in production environments. It combines principles from machine learning, software engineering, and DevOps to ensure the reliability, scalability, and efficiency of ML systems.

MLOps focuses on automating the end-to-end lifecycle of machine learning models, including data preparation, model training, testing, deployment, monitoring, and continuous improvement. It addresses challenges such as model versioning, reproducibility, scalability, monitoring, and collaboration between data scientists, ML engineers, and operations teams.

Here are some popular tools and frameworks commonly used in MLOps:

1. MLflow: An open-source platform for managing the ML lifecycle, including experiment tracking, model packaging, and deployment.

2. Kubeflow: A Kubernetes-native platform for deploying and managing ML workflows, enabling scalable and portable ML pipelines.

3. TensorFlow Extended (TFX): An end-to-end platform for deploying production ML pipelines, including components for data validation, transformation, training, and serving.

4. DVC (Data Version Control): A version control system for ML projects that helps manage datasets, models, and pipelines.

5. Apache Airflow: A platform to programmatically author, schedule, and monitor workflows, often used for orchestrating ML pipelines.

6. Docker: A containerization platform that allows packaging ML models and their dependencies into portable containers for consistent deployment across different environments.

7. Kubernetes: An open-source container orchestration system that automates the deployment, scaling, and management of containerized applications, including ML services.

8. AWS SageMaker: A fully managed platform that provides tools and services for building, training, and deploying ML models at scale.

9. Google Cloud AI Platform: A suite of tools and services for developing, deploying, and managing ML models on Google Cloud.

10. Azure Machine Learning: A cloud-based platform that provides end-to-end ML capabilities, including experiment tracking, model management, and deployment.

These are just a few examples, and the choice of tools depends on your specific requirements, existing infrastructure, and the ML frameworks you are using. It's important to evaluate your needs and consider factors such as scalability, integration with existing systems, ease of use, and community support when selecting MLOps tools for your projects.

In [ ]:
async def main() -> None:

    completion = async_client.messages.stream(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": Prompt,
            }
        ],
        model="claude-3-opus-20240229",
    )
    async with completion as stream:
        async for text in stream.text_stream:
            print(text, end="", flush=True)

# asyncio.run(main())
await main()  # Juypter notebook

Certainly! MLOps, short for Machine Learning Operations, is a set of practices, principles, and tools that aim to streamline the development, deployment, and maintenance of machine learning models in production environments. It combines the best practices from machine learning, DevOps, and software engineering to ensure the reliability, scalability, and efficiency of ML systems.

MLOps focuses on the entire lifecycle of machine learning models, from data collection and preprocessing to model training, validation, deployment, monitoring, and continuous improvement. The goal is to automate and standardize the ML workflow, enabling teams to collaborate effectively, reduce manual efforts, and deliver high-quality models faster.

Here are some popular tools and platforms used in MLOps:

1. TensorFlow Extended (TFX): An end-to-end platform for deploying production ML pipelines. It provides a set of libraries and tools for data validation, preprocessing, model training, serving, and monitorin